In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from plotnine import *
import tqdm
import xswap

In [2]:
density = np.linspace(0.001, 0.8, 10)

rows = list()
for d in tqdm.tqdm_notebook(density):
    G_erdos = nx.generators.fast_gnp_random_graph(1000, d)
    edges_erdos = list(G_erdos.edges)
    original_erdos = set(edges_erdos)
    edges_erdos = list(original_erdos)
    n_original_erdos = len(edges_erdos)
    
    
    G_power = nx.generators.barabasi_albert_graph(1000, int(1000 * d))
    edges_power = list(G_power.edges)
    original_power = set(edges_power)
    edges_power = list(original_power)
    n_original_power = len(original_power)
    
    
    multipliers = np.linspace(0, 10, num=20)
    for m in multipliers:
        perm_erdos, _ = xswap.permute_edge_list(edges_erdos, allow_antiparallel=True, 
                                                allow_self_loops=True, seed=0, multiplier=m)
        percent_swapped_erdos = 1 - len(set(perm_erdos).intersection(original_erdos)) / n_original_erdos
        rows.append((d, m, percent_swapped_erdos, m * n_original_erdos, 'Erdős-Rényi'))
        
        perm_power, _ = xswap.permute_edge_list(edges_power, allow_antiparallel=True, 
                                                allow_self_loops=True, seed=0, multiplier=m)
        percent_swapped_power = 1 - len(set(perm_power).intersection(original_power)) / n_original_power
        rows.append((d, m, percent_swapped_power, m * n_original_power, 'Barabási–Albert'))        

In [3]:
df = pd.DataFrame(rows, columns=['density', 'multiplier', 'percent_swapped', 'num_attempts', 'network_type'])
df.to_csv('percent_swapped.csv', index=False)
df.head()

,density,multiplier,percent_swapped,num_attempts,network_type
0,0.001,0.000000,0.000000,0.000000,Erdős-Rényi
1,0.001,0.000000,0.000000,0.000000,Barabási–Albert
2,0.001,0.526316,0.627753,238.947368,Erdős-Rényi
3,0.001,0.526316,0.623624,525.789474,Barabási–Albert
4,0.001,1.052632,0.859031,477.894737,Erdős-Rényi
